In [63]:
from pulp import *
import os
import numpy as np
import pandas as pd

def read_matrix(path_file):
    satisfactions = []
    resources = []
    capacities = []
    
    with open(path_file, 'r') as f:
        m, n= f.readline().split(' ')[:2]
        m, n = int(m), int(n)
        lines = f.readlines()
        lines = [line.rstrip().split(' ') for line in lines]

    
        for i in range(0, m):
            for j in range(n):
                satisfactions.append(int(lines[i][j]))


        for i in range(m, m + m):
            for j in range(n):
                resources.append(int(lines[i][j]))
        
        for j in range(m):
            capacities.append(int(lines[2*m][j]))
            
    satisfactions = np.array(satisfactions)
    resources = np.array(resources)
    capacities = np.array(capacities)
    
    return m, n, satisfactions, resources, capacities

In [64]:
read_matrix("./entrada1.txt")

(20,
 1600,
 array([-811,  -57,  -14, ...,  -92,  -52,  -13]),
 array([36, 51, 96, ..., 17, 50, 91]),
 array([3244, 3193, 3250, 3247, 3191, 3276, 3150, 3299, 3198, 3179, 3285,
        3228, 3257, 3199, 3196, 3224, 3316, 3273, 3325, 3223]))

In [66]:
ans = []

solver = CPLEX_PY()

prob = LpProblem("desig_gen_problem", LpMaximize)

m, n, sat, res, cap = read_matrix("./entrada1.txt")

N = range(n*m)

x = LpVariable.dicts(
    "x", N, lowBound=0, upBound=1.0
)

prob += lpSum([sat[i] * x[i]  for i in N]), 'objective F'

#   every job can only be picked by one worker
for i in range(n):
    prob += lpSum([x[i + j*n] for j in range(m)]) == 1

#   every worker has to take only the jobs that they can
for i in range(m):
    prob += lpSum([x[i*n + j] * res[i*n + j] for j in range(n)]) <= cap[i]

prob.solve(solver)

solver.buildSolverModel(prob)
solver.solverModel.parameters.timelimit.set(10)

solver.callSolver(prob)
status = solver.findSolutionValues(prob)



# # df_res = pd.DataFrame(res)
# # df_res.to_csv('results_lp.csv')

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 1620 rows, 32000 columns, and 64000 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (17.85 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1620 rows, 32000 columns, and 64000 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (25.10 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.23 sec. (205.09 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0     0      integral     0   -97821.3500   -97821.3500       18    0.00%
Elapsed time = 0.33 sec. (269.86 ticks, tree = 0.00 MB, solutions = 